In [1]:
import pandas as pd
import numpy as np
wd = '/Users/timvigers/Dropbox/Work/CF/Edith Zemanick/CF Nutrition/'

In [2]:
# Import
# Modulator dates
mods = pd.read_excel(wd+"Data_Raw/Report CFTR Modulators_throughDecember2021.xlsx")
mods.sort_values(by=["Patient ID","Earliest Date in Registry"],ascending=True,inplace=True)
mods = mods[mods['Earliest Date in Registry'] != mods['Last Date in Registry']]
# Vitamins
vit = pd.read_excel(wd+"Data_Raw/Report Vitamins_throughDecember2021_revised20220712.xlsx")
# Add FEV information
fev = pd.read_excel(wd+"Data_Raw/Report_PFTs_ThroughDecember2021.xlsx")
fev=fev.rename(columns = {'PatientID':'Patient ID'})
df = pd.merge(vit,fev,how = "outer",on=["Patient ID","Date"])

In [3]:
# Get modulator at each vitamin measure
m = []
for r in range(df.shape[0]):
    # Get ID
    id = df.iloc[r,0]
    # If no mod information, return NA
    if id not in mods["Patient ID"].unique():
        m.append(np.nan)
        continue
    # Get row date
    date = df.iloc[r,2]
    # Get modulator dates by ID
    mod = mods.loc[mods["Patient ID"] == id,:]
    dates = mod.iloc[:,3]
    # Check which dates the row is after
    after = np.where(date > dates)[0]
    # If not after any, return no modulator
    if after.shape[0] == 0: 
        m.append("None")
    else:
        m.append(mod.iloc[max(after),1])
df["Modulator"] = m
# Drop those without modulator info
df.dropna(subset="Modulator",inplace=True)
# Write for checking
df.to_csv("~/t.csv",index=False)